# Imports

In [48]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from transformers import BertTokenizer, BertModel, AutoTokenizer, AutoModel
import torch
import os
import re
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from nltk.corpus import stopwords
import spacy
from nltk.stem import WordNetLemmatizer
import deepl

# File loading

In [49]:
# Load the files 
question_globales_path = 'data/QuestionGlobales.xlsx'
extraction_codebook_path = 'data/Extraction CodeBook - 3. Cleaned.xlsx'
gsb_path = 'data/GSB 2023_V1.xlsx'

# Read the global questions file
df_globales = pd.read_excel(question_globales_path)

# Read all sheets from the codebook file
sheets_codebook = pd.read_excel(extraction_codebook_path, sheet_name=None)

# answers dataframe
gsb_df = pd.read_excel(gsb_path) 


In [50]:
gsb_df.head()

,BFS_2023,Gemeinde_2023,GSB23_Teilnahme,GSB23_Mode,GSB23_StartDate,GSB23_EndDate,GSB23_Progress,GSB23_Duration__in_seconds_,GSB23_Finished,GSB23_RecordedDate,...,gexesvpat,gexespsat,gexeandat,gexeploat,stfussv,orgpart,autonomie,zamitkt,wahlverf,legislative
0,1,Aeugst am Albis,1,0.0,2023-02-16 08:34:08,2023-02-16 10:16:09,100.0,6121.0,1.0,2023-02-16 10:16:10,...,0.000000,0.000000,66.666667,0.000000,1.0,1.0,7.0,3.0,1.0,1.0
1,2,Affoltern am Albis,1,0.0,2023-02-16 08:50:23,2023-02-17 10:17:44,100.0,91641.0,1.0,2023-02-17 10:17:45,...,12.500000,12.500000,50.000000,12.500000,2.0,1.0,5.0,3.0,1.0,1.0
2,3,Bonstetten,1,0.0,2023-03-30 10:16:48,2023-03-30 11:49:16,100.0,5547.0,1.0,2023-03-30 11:49:17,...,16.666667,0.000000,0.000000,66.666667,2.0,1.0,5.0,2.0,1.0,1.0
3,4,Hausen am Albis,1,0.0,2023-02-28 10:27:13,2023-02-28 11:45:36,100.0,4702.0,1.0,2023-02-28 11:45:37,...,14.285714,14.285714,0.000000,42.857143,2.0,1.0,6.0,2.0,1.0,1.0
4,5,Hedingen,1,0.0,2023-02-20 14:13:19,2023-02-20 14:34:49,40.0,1289.0,0.0,NaT,...,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN


## Start of data manipulation before NLP preprocessing pipeline

In [51]:
# create a dataframe for each year
years = ['1988', '1994', '1998', '2005', '2009', '2017', '2023']
dataframes = {}

# choose the columns to keep
columns_to_keep = ['code', 'num_question', 'year', 'label', 'type', 'format']


for year in years:
    if year in sheets_codebook:  # ensure the sheet exists
        # each sheet is a DataFrame
        df = sheets_codebook[year]
        # keep only the columns we need (defined in columns_to_keep)
        dataframes[f'main_questions_{year}'] = df[columns_to_keep]
    else:
        print(f"Sheet for year {year} not found.")

# define the dataframes
main_questions_1988 = dataframes.get('main_questions_1988', pd.DataFrame())
main_questions_1994 = dataframes.get('main_questions_1994', pd.DataFrame())
main_questions_1998 = dataframes.get('main_questions_1998', pd.DataFrame())
main_questions_2005 = dataframes.get('main_questions_2005', pd.DataFrame())
main_questions_2009 = dataframes.get('main_questions_2009', pd.DataFrame())
main_questions_2017 = dataframes.get('main_questions_2017', pd.DataFrame())
main_questions_2023 = dataframes.get('main_questions_2023', pd.DataFrame())

main_questions_2009.head()

,code,num_question,year,label,type,format
0,GSB09_1,1,2009,"Es kann Probleme geben, welche die Gemei...",NaN,NaN
1,GSB09_1_1,1,2009,Unterstützung und Betreuung älterer Menschen,NaN,NaN
2,GSB09_1_2,1,2009,Jugendfragen,NaN,NaN
3,GSB09_1_3,1,2009,familienergänzende Kinderbetreuung,NaN,NaN
4,GSB09_1_4,1,2009,Unterstützung und Betreuung von Arbeitslosen,NaN,NaN


## Sheet filtering for num_question

In [52]:
# tailor made data cleaning for the old years sheets 
def filter_num_question_old(df):
    # delete rows with empty num_question
    df = df[df['num_question'].notna()].copy()  

    # convert num_question to string
    df.loc[:, 'num_question'] = df['num_question'].astype(str)  

    # some tailor made filtering --> keep only rows with num_question that are digits or contain 'a' or 'A' 
    df = df[df['num_question'].str.match(r'^\d+$|.*[aA].*')]
    
    # keep only the first occurence of each num_question
    df = df.drop_duplicates(subset='num_question', keep='first')

    # delete the a or A from the num_question
    df['num_question'] = df['num_question'].str.replace('a', '', regex=False).str.replace('A', '', regex=False)

    # keep only the first occurence of each num_question
    df = df.drop_duplicates(subset='num_question', keep='first')

    return df

In [53]:
# tailor made data cleaning for the new years sheets 
def filter_num_question_new(df):
    # delete rows with empty num_question
    df = df[df['num_question'].notna()].copy()  
    # keep only the first occurence of each num_question
    df = df.drop_duplicates(subset='num_question', keep='first')
    return df

In [54]:
main_questions_1988 = filter_num_question_old(main_questions_1988)
main_questions_1994 = filter_num_question_old(main_questions_1994)
main_questions_1998 = filter_num_question_old(main_questions_1998)
main_questions_2005 = filter_num_question_old(main_questions_2005)
main_questions_2009 = filter_num_question_old(main_questions_2009)
main_questions_2017 = filter_num_question_new(main_questions_2017)
main_questions_2017 = main_questions_2017[main_questions_2017['num_question'] != 'Q1']
main_questions_2023 = filter_num_question_new(main_questions_2023)


In [55]:
main_questions_1988.head()

,code,num_question,year,label,type,format
0,GSB88_1a,1,1988,Postleitzahl der Gemeinde,NaN,NaN
5,GSB88_2a,2,1988,Wieviele Einwohner umfasst ihre Gemeinde,NaN,NaN
7,GSB88_3,3,1988,Welche Beschreibung trifft auf Ihre Gemeinde a...,NaN,NaN
8,GSB88_4,4,1988,Welche dieser Bezeichnungen treffen am ehesten...,NaN,NaN
20,GSB88_5a,5,1988,Wenn man das Zusammenleben in einer Gemeinde b...,NaN,NaN


In [56]:
dataframes_to_merge = [
    main_questions_1988,
    main_questions_1994,
    main_questions_1998,
    main_questions_2005,
    main_questions_2009,
    main_questions_2017,
    main_questions_2023
]

# merge all the dataframes
merged_df = pd.concat(dataframes_to_merge, ignore_index=True)

In [57]:
merged_df.head()

,code,num_question,year,label,type,format
0,GSB88_1a,1,1988,Postleitzahl der Gemeinde,NaN,NaN
1,GSB88_2a,2,1988,Wieviele Einwohner umfasst ihre Gemeinde,NaN,NaN
2,GSB88_3,3,1988,Welche Beschreibung trifft auf Ihre Gemeinde a...,NaN,NaN
3,GSB88_4,4,1988,Welche dieser Bezeichnungen treffen am ehesten...,NaN,NaN
4,GSB88_5a,5,1988,Wenn man das Zusammenleben in einer Gemeinde b...,NaN,NaN


## Start of text preprocessing

In [58]:
!python -m spacy download de_core_news_sm

     ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
     --------------------------------------- 0.0/14.6 MB 660.6 kB/s eta 0:00:23
     ---------------------------------------- 0.2/14.6 MB 1.8 MB/s eta 0:00:08
     - -------------------------------------- 0.5/14.6 MB 3.9 MB/s eta 0:00:04
     -- ------------------------------------- 0.8/14.6 MB 4.5 MB/s eta 0:00:04
     --- ------------------------------------ 1.1/14.6 MB 5.5 MB/s eta 0:00:03
     ---- ----------------------------------- 1.5/14.6 MB 5.9 MB/s eta 0:00:03
     ----- ---------------------------------- 1.8/14.6 MB 5.8 MB/s eta 0:00:03
     ------ --------------------------------- 2.2/14.6 MB 6.4 MB/s eta 0:00:02
     ------- -------------------------------- 2.6/14.6 MB 6.6 MB/s eta 0:00:02
     -------- ------------------------------- 3.0/14.6 MB 6.7 MB/s eta 0:00:02
     --------- ------------------------------ 3.3/14.6 MB 6.8 MB/s eta 0:00:02
     ---------- ----------------------------- 3.7/14.6 MB 

In [59]:
nlp = spacy.load('de_core_news_sm')

def preprocess_text(text):
    # 1. convert text to lowercase
    text = text.lower()
    
    # 2. delete punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # 3. delete extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 4. lemmatization + tokenization + remove stopwords
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop] 
    
    return tokens  

# apply the preprocess_text function to the 'label' column
tqdm.pandas() 
merged_df['tokens'] = merged_df['label'].progress_apply(preprocess_text)


100%|██████████| 563/563 [00:02<00:00, 194.28it/s]


In [60]:
merged_df.head()

,code,num_question,year,label,type,format,tokens
0,GSB88_1a,1,1988,Postleitzahl der Gemeinde,NaN,NaN,"[Postleitzahl, Gemeinde]"
1,GSB88_2a,2,1988,Wieviele Einwohner umfasst ihre Gemeinde,NaN,NaN,"[wieviele, Einwohner, umfassen, Gemeinde]"
2,GSB88_3,3,1988,Welche Beschreibung trifft auf Ihre Gemeinde a...,NaN,NaN,"[Beschreibung, treffen, Gemeinde]"
3,GSB88_4,4,1988,Welche dieser Bezeichnungen treffen am ehesten...,NaN,NaN,"[Bezeichnung, Treffen, eher, Gemeinde]"
4,GSB88_5a,5,1988,Wenn man das Zusammenleben in einer Gemeinde b...,NaN,NaN,"[zusammenleben, Gemeinde, betrachten, extrem, ..."


# NLP pipeline: 

## Transformer version: Bert

In [61]:
model_name = 'dbmdz/bert-base-german-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [62]:
def get_embeddings(labels):
    embeddings = []
    for label in tqdm(labels, desc='Generating embeddings'):
        inputs = tokenizer(label, return_tensors='pt', padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            embedding = outputs.last_hidden_state[:, 0, :].numpy()
            embeddings.append(embedding)
    return np.vstack(embeddings)


In [63]:
# obtain the tokens from the merged DataFrame
labels = merged_df['tokens'].apply(lambda x: ' '.join(x)).unique()  # convert the list of tokens to a string

# obtain the embeddings for the tokens
embeddings = get_embeddings(labels)

# use cosine similarity to calculate the similarity matrix
similarity_matrix = cosine_similarity(embeddings)

# create a DataFrame from the similarity matrix
similarity_df = pd.DataFrame(similarity_matrix, index=labels, columns=labels)

# set the threshold for similarity --> here 0.97 but can be changed
threshold = 0.97

similar_pairs = []

# iterate over the similarity matrix to identify similar pairs
for i in tqdm(range(len(labels)), desc='Identifying similar pairs'):
    for j in range(i + 1, len(labels)):  
        if similarity_matrix[i][j] >= threshold:
            similar_pairs.append((labels[i], labels[j]))

# create a new DataFrame to store the combined rows
question_globale_new = pd.DataFrame(columns=['id', 'label', 'year', 'code'])

id_counter = 1

for label1, label2 in similar_pairs:
    # add a row for each pair of similar tokens
    temp_df = merged_df[(merged_df['tokens'].apply(lambda x: ' '.join(x)) == label1) | 
                        (merged_df['tokens'].apply(lambda x: ' '.join(x)) == label2)]
    combined_row = {
        'id': id_counter,  # ad id for each combined row
        'label': f"{label1}; {label2}",
        'year': '; '.join(temp_df['year'].astype(str)),
        'code': '; '.join(temp_df['code'])
    }

    question_globale_new = pd.concat([question_globale_new, pd.DataFrame([combined_row])], ignore_index=True)

    id_counter += 1


Identifying similar pairs: 100%|██████████| 514/514 [00:00<00:00, 2040.92it/s]


In [64]:
question_globale_new.head()

,id,label,year,code
0,1,wieviele administrativ tätig angestellter Beam...,1988; 1994,GSB88_8; GSB94_39a
1,2,stark persönlich Einschätzung allgemein einflu...,1988; 2005; 2009,GSB88_42; GSB05_30; GSB09_29
2,3,steuerfuss Steueranlage Gemeinde Vergleich 199...,1994; 1998; 2005,GSB94_15; GSB98_11; GSB05_3
3,4,steuerfuss Steueranlage Gemeinde Vergleich 199...,1994; 1998; 2009,GSB94_15; GSB98_11; GSB09_2
4,5,real Ertrag Einkommen Vermögenssteuer Gemeinde...,1994; 1998; 2005,GSB94_16; GSB98_12; GSB05_4


In [65]:
question_globale_new.shape

(43, 4)

In [66]:
question_globale_new[question_globale_new['id'] == 1].label.values[0]

'wieviele administrativ tätig angestellter Beamter Lehrlinge Gemeindeverwaltung beschäftigen Personal gemeindeeigen Werk anstalen Bau Strassenarbeiter abwart Polizei; wieviele administrativ tätig angestellter Beamter Lehrlinge Gemeindeverwaltung beschäftigen respektive 10 beschäftigen Personal gemeindeeigen Werk anstalen Bau Strassenarbeiter abwart Polizei'

## Transformer version: DSBSAN

In [67]:
# obtain the tokens from the merged DataFrame
labels = merged_df['tokens'].apply(lambda x: ' '.join(x)).unique()

# embedding model
def get_embeddings(labels):
    embeddings = []
    for label in tqdm(labels, desc="embedding computation", unit="label"):
        inputs = tokenizer(label, return_tensors='pt', padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            embedding = outputs.last_hidden_state[:, 0, :].numpy()  
            embeddings.append(embedding)
    return np.vstack(embeddings)

# obtain unique labels
labels = merged_df['label'].unique()

# obtain embeddings for the labels
embeddings = get_embeddings(labels)
print("Shape of embeddings:", embeddings.shape)

# use cosine similarity to calculate the similarity matrix
distance_matrix = cosine_distances(embeddings)

# apply DBSCAN clustering
dbscan = DBSCAN(eps=0.05, min_samples=3, metric='precomputed')  # use precomputed to pass the distance matrix
clusters = dbscan.fit_predict(distance_matrix)

# create a dictionary to map labels to indices
label_to_index = {label: index for index, label in enumerate(merged_df['label'].unique())}

# create a Series to store the clusters
cluster_series = pd.Series(-1, index=merged_df.index)

# assign the clusters to the labels
for label, cluster_id in zip(labels, clusters):
    index = label_to_index[label]
    cluster_series.iloc[index] = cluster_id

# add the clusters to the merged DataFrame
merged_df['cluster'] = cluster_series

# create a new DataFrame to store the combined rows
question_globale_new = pd.DataFrame(columns=['id', 'label', 'year', 'code'])

# fill the new DataFrame with the combined rows
id_counter = 1
for cluster_id in set(merged_df['cluster']):
    if cluster_id != -1:  
        temp_df = merged_df[merged_df['cluster'] == cluster_id]
        if not temp_df.empty:
            combined_row = {
                'id': id_counter,
                'label': '; '.join(temp_df['label']),
                'year': '; '.join(temp_df['year'].astype(str)),
                'code': '; '.join(temp_df['code'])
            }
            question_globale_new = pd.concat([question_globale_new, pd.DataFrame([combined_row])], ignore_index=True)
            id_counter += 1


embedding computation: 100%|██████████| 549/549 [00:25<00:00, 21.64label/s]

Shape of embeddings: (549, 768)


In [68]:
question_globale_new.shape

(18, 4)

In [69]:
question_globale_new.head()

,id,label,year,code
0,1,Wenn in Ihrer Gemeinde das Amt des Schreibers ...,1988; 1988; 1988; 1988; 1988; 1988; 1988; 1988...,GSB88_9; GSB88_11; GSB88_21a; GSB88_22a; GSB88...
1,2,Wurde von einer Partei bei den letzten allgeme...,1988; 1988; 2005; 2009; 2009; 2017,GSB88_27a; GSB88_28a; GSB05_41a1; GSB09_37; GS...
2,3,Wurden in den letzten 5 Jahren Entscheid...,1988; 1994; 2005,GSB88_38; GSB94_30a; GSB05_22a
3,4,Nach welchem Wahlverfahren wird das Parlament ...,1988; 1998; 2017; 2017; 2017,GSB88_39; GSB98_40; GSB17_Q27; GSB17_Q135_2; G...
4,5,Wie würden Sie die Zusammenarbeit zw...,1988; 1988; 1994; 1994; 1994; 1994; 1998; 1998...,GSB88_47; GSB88_57; GSB94_13a16; GSB94_14a; GS...


In [70]:
question_globale_new[question_globale_new['id'] == 1].label.values[0]

'Wenn in Ihrer Gemeinde das Amt des Schreibers (oder eines anderen Chefbeamten)    neu    besetzt    werden    soll:    wird    da    auf    die\nParteimitgliedschaft der Bewerber geachtet?; Ist   der   Präsident   (LU:   Ammann)   Ihrer   Gemeinde   vollamtlich, halbamtlich oder nebenamtlich tätig?; Sind  Selbständigerwerbende  im Gemeindevorstand  (inkl.  Präsident)\nvertreten?; Befinden sich unter den Vorstandsmitgliedern (inkl. Präsident) auch Mandatsträger auf kantonaler oder nationaler Ebene?; Wieviele Kandidaten haben sich insgesamt bei den letzten ordent- lichen  Wahlen  um  Sitze  im  Gemeindevorstand  beworben,  und  von\nwelcher Partei (Partei und Anzahl) wurden sie gestellt?; Wie oft kommt es vor, dass Vorstandsmitglieder, die derselben Partei angehören,    bei    Sachentscheiden    gegensätzliche    Auffassungen\nvertreten?; Gab   es   bei   den   letzten   oder   vorletzten   allgemeinen   Wahlen amtierende Vorstandsmitglieder, die erneut kandidierten, aber nicht\nwiederg

## TF-IDF version

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# obtain the tokens from the merged DataFrame
labels = merged_df['tokens'].apply(lambda x: ' '.join(x)).unique()

# create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(labels)

# calculate the cosine similarity matrix
similarity_matrix = cosine_similarity(tfidf_matrix)

# create a DataFrame from the similarity matrix
similarity_df = pd.DataFrame(similarity_matrix, index=labels, columns=labels)

# define the threshold for similarity, here 0.8 seems to be a good value but can be changed 
threshold = 0.8

similar_pairs = []

# identify similar pairs
for i in tqdm(range(len(labels)), desc='Identifying similar pairs'):
    for j in range(i + 1, len(labels)):
        if similarity_matrix[i][j] >= threshold:
            similar_pairs.append((labels[i], labels[j]))

# create a new DataFrame to store the combined rows
question_globale_new = pd.DataFrame(columns=['id', 'label', 'year', 'code'])

id_counter = 1

for label1, label2 in similar_pairs:
    # add a row for each pair of similar tokens
    temp_df = merged_df[(merged_df['tokens'].apply(lambda x: ' '.join(x)) == label1) | 
                        (merged_df['tokens'].apply(lambda x: ' '.join(x)) == label2)]
    combined_row = {
        'id': id_counter,  
        'label': f"{label1}; {label2}",
        'year': '; '.join(temp_df['year'].astype(str)),
        'code': '; '.join(temp_df['code'])
    }

    question_globale_new = pd.concat([question_globale_new, pd.DataFrame([combined_row])], ignore_index=True)

    id_counter += 1


Identifying similar pairs: 100%|██████████| 514/514 [00:00<00:00, 9894.59it/s]


In [72]:
question_globale_new.shape

(73, 4)

In [73]:
question_globale_new.head()

,id,label,year,code
0,1,wieviele administrativ tätig angestellter Beam...,1988; 1994,GSB88_8; GSB94_39a
1,2,politisch Auseinandersetzung Gemeinde Auge hal...,1988; 1994; 2005,GSB88_52; GSB94_21; GSB05_29
2,3,Wahlbeteiligung Gemeinde; Wahlbeteiligung Geme...,1988; 1998; 2017,GSB88_55; GSB98_43; GSB17_Q35
3,4,Gemeinde unterschiedlich Masse bestimmt gesell...,1994; 1998; 2005,GSB94_5; GSB98_3; GSB05_1
4,5,Problem Gemeinde aufgrund hoch Arbeitsbelastun...,1994; 1998; 2005; 2009,GSB94_8; GSB98_5; GSB05_2; GSB09_1


In [74]:
question_globale_new[question_globale_new['id'] == 5].label.values[0]

'Problem Gemeinde aufgrund hoch Arbeitsbelastung mangelnd Fachkompetenz angemessen Weise bewältigen Bereich Gemeinde leistungsgrenzen lg sichtbar; Problem geben Gemeinde aufgrund hoch Arbeitsbelastung mangelnd Fachkompetenz angemessen Weise bewältigen Gemeinde leistungsgrenzen lg folgend aufgabenbereich sichtbar'

### Here the results seems better --> use the TF-IDF version 

# CSV extraction of the global questions

In [75]:
quest_glob_columns = [
    'label', 'code_first_question','text_de', 'text_fr', 'text_it', 'text_ro', 'text_en',
    'category_label', 'category_text_de', 'category_text_fr',
    'category_text_it', 'category_text_ro', 'category_text_en',
    'options_value', 'options_label'
]

code_to_token = {}

for index, row in merged_df.iterrows():
    code = row['code']
    tokens = row['tokens']
    if code not in code_to_token and tokens:  
        code_to_token[code] = tokens[0]


print(code_to_token)

{'GSB88_1a': 'Postleitzahl', 'GSB88_2a': 'wieviele', 'GSB88_3': 'Beschreibung', 'GSB88_4': 'Bezeichnung', 'GSB88_5a': 'zusammenleben', 'GSB88_6': 'Gemeinde', 'GSB88_7': 'wieviele', 'GSB88_8': 'wieviele', 'GSB88_9': 'Gemeinde', 'GSB88_10a': 'Gemeindeschreiber', 'GSB88_11': 'Präsident', 'GSB88_12a': 'Präsident', 'GSB88_13': 'Partei', 'GSB88_14a': 'Kandidat', 'GSB88_15': 'Ort', 'GSB88_16': 'wahlverfahren', 'GSB88_17a': 'Anzahl', 'GSB88_19a': 'Frau', 'GSB88_20a': 'finden', 'GSB88_21a': 'Selbständigerwerbend', 'GSB88_22a': 'befinden', 'GSB88_23a': 'wieviele', 'GSB88_24': 'möchten', 'GSB88_25': 'vorstandsmitglied', 'GSB88_26a': 'letzter', 'GSB88_27a': 'Partei', 'GSB88_28a': 'Partei', 'GSB88_29': 'letzter', 'GSB88_30': 'Gemeinde', 'GSB88_31a': 'wieviele', 'GSB88_32': 'häufig', 'GSB88_33a': 'bestehen', 'GSB88_34': 'Gemeinde', 'GSB88_35': 'Gemeinde', 'GSB88_36a': 'wievieler', 'GSB88_37': 'haufig', 'GSB88_38': 'letzter', 'GSB88_39': 'wahlverfahren', 'GSB88_40a': 'festgelegt', 'GSB88_41': 'Gemein

In [76]:
# create a new DataFrame to store the final results
df_fin = pd.DataFrame(columns=quest_glob_columns)

# fill the new DataFrame with the combined rows
for index, row in question_globale_new.iterrows():
    code_first_question = row['code'].split('; ')[0]  # use the first code in 'code'
    text_de_first_question = row['label'].split('; ')[0]
    first_token = code_to_token.get(code_first_question, '')  # use the token corresponding to the first code

    # create a new row with the required columns
    new_row = {
        'label': first_token,  # use the first code in 'label'
        'code_first_question': code_first_question,  # put the first code in 'code_first_question'
        'text_de': text_de_first_question,  # use the first label in 'text_de'
        'text_fr': '',  # to be filled with deepl API 
        'text_it': '', # to be filled with deepl API 
        'text_ro': '', # to be filled with deepl API 
        'text_en': '', # to be filled with deepl API 
        'category_label': '',
        'category_text_de': '',
        'category_text_fr': '',
        'category_text_it': '',
        'category_text_ro': '',
        'category_text_en': '',
        'options_value': '',
        'options_label': ''
    }

    # add the new row to the final DataFrame
    df_fin = pd.concat([df_fin, pd.DataFrame([new_row])], ignore_index=True)


df_fin.head()


,label,code_first_question,text_de,text_fr,text_it,text_ro,text_en,category_label,category_text_de,category_text_fr,category_text_it,category_text_ro,category_text_en,options_value,options_label
0,wieviele,GSB88_8,wieviele administrativ tätig angestellter Beam...,,,,,,,,,,,,
1,politisch,GSB88_52,politisch Auseinandersetzung Gemeinde Auge hal...,,,,,,,,,,,,
2,Wahlbeteiligung,GSB88_55,Wahlbeteiligung Gemeinde,,,,,,,,,,,,
3,Gemeinde,GSB94_5,Gemeinde unterschiedlich Masse bestimmt gesell...,,,,,,,,,,,,
4,Problem,GSB94_8,Problem Gemeinde aufgrund hoch Arbeitsbelastun...,,,,,,,,,,,,


In [77]:
df_fin[df_fin['code_first_question'] == 'GSB94_5'].text_de.values

array(['Gemeinde unterschiedlich Masse bestimmt gesellschaftlich Entwicklung betreffen nachfolgend Reihe Entwicklung auflisten bitte geben Ausmass Gemeinde betreffen'],
      dtype=object)

In [78]:
gsb_columns = gsb_df.columns

# fill the 'options_value' column in df_fin
for index, row in df_fin.iterrows():
    # get the code of the first question
    question_code = row['code_first_question']
    
    # check if the question code exists in the 'GSB 2023_V1' DataFrame
    if question_code in gsb_columns:
        # extract the unique values from the column
        unique_values = gsb_df[question_code].dropna().unique()
        
        # if there are unique values, join them 
        if len(unique_values) > 0:
            options_value = ";".join(map(str, unique_values))
        else:
            options_value = ''
        
        # fill the options_value in the final DataFrame
        df_fin.at[index, 'options_value'] = options_value
    else:
        # if the question code does not exist in the 'GSB 2023_V1' DataFrame, fill an empty string
        # to be adapted when other version of the answers than 2023 will be available
        df_fin.at[index, 'options_value'] = ''


In [79]:
df_fin.head(100)

,label,code_first_question,text_de,text_fr,text_it,text_ro,text_en,category_label,category_text_de,category_text_fr,category_text_it,category_text_ro,category_text_en,options_value,options_label
0,wieviele,GSB88_8,wieviele administrativ tätig angestellter Beam...,,,,,,,,,,,,
1,politisch,GSB88_52,politisch Auseinandersetzung Gemeinde Auge hal...,,,,,,,,,,,,
2,Wahlbeteiligung,GSB88_55,Wahlbeteiligung Gemeinde,,,,,,,,,,,,
3,Gemeinde,GSB94_5,Gemeinde unterschiedlich Masse bestimmt gesell...,,,,,,,,,,,,
4,Problem,GSB94_8,Problem Gemeinde aufgrund hoch Arbeitsbelastun...,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,Personalbestand,GSB17_Q53,Personalbestand Stellenprozent letzter 10 verä...,,,,,,,,,,,,
69,arbeiten,GSB17_Q58,arbeiten derder Gemeindepräsidentin aktiv Verw...,,,,,,,,,,,,
70,operativ,GSB17_Q62,operativ Leitung Gemeindeverwaltung zuständig,,,,,,,,,,,,
71,hoch,GSB17_Q66,hoch abgeschlossen Ausbildung,,,,,,,,,,,,


### Results extraction

In [80]:
df_fin.to_csv('data/QuestionGlobales_NLP.csv', index=False, encoding='utf-8')